In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [2]:
resnet = torchvision.models.resnet18(pretrained=True)
vgg = torchvision.models.vgg16(pretrained=True)
efficient = torchvision.models.efficientnet_b0(pretrained=True)

/home/sentooooooon/pytorch-learnings/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sentooooooon/pytorch-learnings/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/sentooooooon/pytorch-learnings/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1

In [3]:
#結合層の出力数
num_classes = 10

#モデルの出力数を調整する
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, num_classes)

efficient.classifier[1] = nn.Linear(efficient.classifier[1].in_features, num_classes)

In [4]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), #左右反転させてバリエーションを増やす
    transforms.RandomRotation(10), #
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
import torch.utils.data.dataloader


train_data = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    transform=transform,
    download = True
)

test_data =  torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    transform=transform,
    download = True
)


BATCH_SIZE = 16
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

In [6]:
#損失関数と最適化関数

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

In [7]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU使用可否を確認

resnet.to(device)  # モデルをデバイスに移動

for epoch in range(num_epochs):
    
    ### 1️⃣ 学習フェーズ (Training)
    resnet.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # GPUに転送
        
        optimizer.zero_grad()  # 勾配を初期化
        outputs = resnet(images)  # 順伝播
        loss = criterion(outputs, labels)  # 損失計算
        loss.backward()  # 逆伝播
        optimizer.step()  # パラメータ更新
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)  # エポックごとの平均損失

    ### 2️⃣ 評価フェーズ (Validation)
    resnet.eval()  # 評価モード
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():  # 勾配計算をオフ
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = resnet(images)
            loss = criterion(outputs, labels)  # 損失計算
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)  # 最大確率のクラスを取得
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(test_loader)
    accuracy = 100 * correct / total  # 精度（%）

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, "
          f"Accuracy: {accuracy:.2f}%")


KeyboardInterrupt: 